In [2]:
import sys
import os

# Assuming the package is one directory above the current working directory
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from diffUV import dyned_eul, dyn_body, kin, dyned_quat
from diffUV.utils.symbols import *
from diffUV.utils.quaternion_ops import euler2q, q2euler
from casadi import *
from blue_rov import Params as ps
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

In [3]:
r_g = vertcat(x_g, y_g, z_g) # center of gravity wrt body origin
r_b = vertcat(x_b, y_b, z_b) # center of buoyancy wrt body origin 
I_o = vertcat(I_x, I_y, I_z,I_xz) # rigid body inertia wrt body origin

decoupled_added_m = vertcat(X_du, Y_dv, Z_dw, K_dp, M_dq, N_dr) # added mass in diagonals
coupled_added_m =  vertcat(X_dq, Y_dp, N_dp, M_du, K_dv) # effective added mass in non diagonals

linear_dc = vertcat(X_u, Y_v, Z_w, K_p,  M_q, N_r) # linear damping coefficients
quadratic_dc = vertcat(X_uu, Y_vv, Z_ww, K_pp, M_qq, N_rr) # quadratic damping coefficients

n0 = vertcat(n, dn) # state variables wrt ned
x_nb # velocity state variables wrt body

SX([u, v, w, p, q, r])

In [4]:
# body representaion
uv_body = dyn_body()


Kinematics = kin()
Jq_ = Kinematics.Jq
J_ = Kinematics.J

In [4]:
Jq_dot = Kinematics.Jq_dot
v_ddot = uv_body.body_forward_dynamics()

xd = Jq_@x_nb
ode_xdd = v_ddot

ode = vertcat(xd, ode_xdd) #the complete ODE vector
f_ode = Function('Odefunc', [m, W, B, r_g, r_b, I_o,
                           decoupled_added_m, coupled_added_m,
                           linear_dc, quadratic_dc,
                           x_nb, n, uq, tau_b], [ode])

n_arg = vertcat(p_n, q2euler(uq))
f_ode_simplify = f_ode(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m,
                       ps.coupl_added_m, ps.linear_dc, ps.quadratic_dc, x_nb,  n_arg, uq,  tau_b)

f_ode_simplify_func = Function('f_ode_simplify', [x_nb , uq, tau_b], [f_ode_simplify]) ## RETURNS vertcat(p_n ,uq , x_nb)
# f_ode_simplify_func


T = 10 # time horizon
N = 1600 # number of control intervals

# integrator to discretize the system
dae = {'x':vertcat(p_n, uq, x_nb), 'p':tau_b, 'ode': f_ode_simplify_func(x_nb ,uq, tau_b)}
intg = integrator('intg', 'rk', dae, {'tf':T/N, 'number_of_finite_elements':100, 'simplify':True})

xS0 = vertcat(p_n, uq, x_nb)
# print(xS0)
res = intg(x0=xS0,p=tau_b) #evaluate with symbols
x_next = res['xf']
x_next[3:7] = x_next[3:7]/sqrt(x_next[3:7].T@x_next[3:7])  #quaternions requires normalization

x_next[9] = if_else(x_next[2] < 0, 0,  x_next[9]) # if vehicle on surface, no more up motion
x_next[2] = if_else(x_next[2] < 0, 0,  x_next[2]) # if vehicle on surface, keep on surface and not go up

CasADi - 2024-06-19 19:13:18 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:521]


In [5]:
# Simplify API to (x,u)->(x_next)
F = Function('Vnext',[xS0,tau_b],[x_next])